In [1]:

# Configure Drive or Jupyter notebook -- only runs when first loaded
if "CONFIG_DONE" not in globals():
    # Need to mount drive and clone repo to access data and functions
    try:
        from google.colab import drive  # type: ignore

        IN_COLAB = True

        # clone repo
        !git clone https://github.com/doctorsmylie/mtg-draft-agent
        %cd mtg-draft-agent

    except ModuleNotFoundError:
        IN_COLAB = False

    # Finish configuration -- also configures notebook outside of Colab
    %run "project_path.ipynb"
else:
    print("Config done already")

Cloning into 'mtg-draft-agent'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 79 (delta 33), reused 52 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 181.35 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/mtg-draft-agent
Starting config...
Running in Colab? Yes

Configuring Google Colab...
Mounting Drive...
Mounted at /content/mtg-draft-agent/drive
BASE_PATH =  /content/mtg-draft-agent
DATA_FOLDER = /content/mtg-draft-agent/drive/MyDrive/Erdos25/MTGdraft
BASE_PATH == os.getcwd(): True

Configuration done


# Testing deck evaluation metrics

Here, we search for suitable metrics with which to evaluate generated decks.

In [2]:
import zipfile
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# For managing paths
import pathlib

First, we import the game data.

In [3]:
folder='drive/MyDrive/MTGdraft'
expansion = 'DSK'
gamefilename ='game_data_public.DSK.PremierDraft.csv.gz'
game_file = pathlib.Path(folder, expansion, gamefilename)

gamedata=pd.read_csv(game_file,compression='gzip',nrows=20000)

/tmp/ipython-input-3-1395837299.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  gamedata=pd.read_csv(game_file,compression='gzip',nrows=20000)


In [4]:
gamedata.describe()

,build_index,match_number,game_number,opp_rank,num_mulligans,opp_num_mulligans,num_turns,opening_hand_Abandoned Campground,drawn_Abandoned Campground,tutored_Abandoned Campground,...,tutored_Withering Torment,deck_Withering Torment,sideboard_Withering Torment,"opening_hand_Zimone, All-Questioning","drawn_Zimone, All-Questioning","tutored_Zimone, All-Questioning","deck_Zimone, All-Questioning","sideboard_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket
count,20000.000000,20000.000000,20000.0,0.0,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.0,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.187400,3.772950,1.0,NaN,0.112600,0.122300,8.974200,0.022750,0.027100,0.000150,...,0.000400,0.095350,0.012700,0.010050,0.016450,0.0,0.058900,0.006800,149.594150,0.537583
std,0.465609,2.166364,0.0,NaN,0.331703,0.343873,2.505703,0.151108,0.166934,0.012247,...,0.019996,0.330097,0.113311,0.101733,0.130691,0.0,0.252456,0.082183,146.991312,0.064872
min,0.000000,1.000000,1.0,NaN,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000
25%,0.000000,2.000000,1.0,NaN,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.480000
50%,0.000000,3.000000,1.0,NaN,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.540000
75%,0.000000,5.000000,1.0,NaN,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.600000
max,4.000000,9.000000,1.0,NaN,3.000000,3.000000,26.000000,2.000000,2.000000,1.000000,...,1.000000,3.000000,2.000000,2.000000,2.000000,0.0,2.000000,1.000000,500.000000,0.740000


Now, we remove entries from users with fewer than 10 total games played.

In [5]:
df_filtered = gamedata.loc[gamedata['user_n_games_bucket']>=10]
df_filtered.describe()

,build_index,match_number,game_number,opp_rank,num_mulligans,opp_num_mulligans,num_turns,opening_hand_Abandoned Campground,drawn_Abandoned Campground,tutored_Abandoned Campground,...,tutored_Withering Torment,deck_Withering Torment,sideboard_Withering Torment,"opening_hand_Zimone, All-Questioning","drawn_Zimone, All-Questioning","tutored_Zimone, All-Questioning","deck_Zimone, All-Questioning","sideboard_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket
count,19985.000000,19985.00000,19985.0,0.0,19985.000000,19985.000000,19985.000000,19985.000000,19985.000000,19985.000000,...,19985.000000,19985.000000,19985.000000,19985.000000,19985.000000,19985.0,19985.000000,19985.000000,19985.000000,19985.000000
mean,0.187541,3.77383,1.0,NaN,0.112534,0.122342,8.973530,0.022567,0.027070,0.000150,...,0.000400,0.095422,0.012710,0.010058,0.016462,0.0,0.058944,0.006805,149.703277,0.537788
std,0.465756,2.16662,0.0,NaN,0.331638,0.343931,2.505848,0.150530,0.166853,0.012251,...,0.020004,0.330210,0.113353,0.101770,0.130740,0.0,0.252545,0.082214,146.992457,0.064344
min,0.000000,1.00000,1.0,NaN,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,10.000000,0.160000
25%,0.000000,2.00000,1.0,NaN,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.480000
50%,0.000000,3.00000,1.0,NaN,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.540000
75%,0.000000,5.00000,1.0,NaN,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,100.000000,0.600000
max,4.000000,9.00000,1.0,NaN,3.000000,3.000000,26.000000,2.000000,2.000000,1.000000,...,1.000000,3.000000,2.000000,2.000000,2.000000,0.0,2.000000,1.000000,500.000000,0.740000


Now, we split the data into training and validation sets.

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor



In [7]:
df_train, df_test = train_test_split(df_filtered,test_size=0.20,random_state=330)
targets_train = df_train['user_game_win_rate_bucket']
features_train = df_train.filter(regex='^deck')
targets_test = df_test['user_game_win_rate_bucket']
features_test = df_test.filter(regex='^deck')

In [11]:
rf = RandomForestRegressor(n_estimators=100,max_depth=8)
rf.fit(features_train,targets_train)

RandomForestRegressor(max_depth=8)

In [12]:
y_pred = rf.predict(features_test)
root_mean_squared_error(targets_test,y_pred)

0.05696595515783297

In [ ]:
ests = [100,200]
depths = range(6,9)
grid_search = GridSearchCV(RandomForestRegressor(), param_grid={'n_estimators': ests, 'max_depth': depths},
    scoring='neg_mean_squared_error')

grid_search.fit(features_train, targets_train)

In [ ]:
grid_search.best_params_